In [1]:
import os
import time
import pickle
from os.path import join, exists, basename
from os import listdir, makedirs
from datetime import datetime
from google import genai
from google.genai import types
from openai import OpenAI, AsyncOpenAI
import requests
import json
from pydantic import BaseModel, Field
from crawl4ai import *
from pydantic_ai.result import RunResult
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.exceptions import UsageLimitExceeded
from pydantic_ai.usage import UsageLimits
from rich import print as rprint
from rich.console import Console
from rich.markdown import Markdown
from queue import Queue, Empty
from dataclasses import dataclass, field
from uuid import UUID, uuid4
from typing import Dict, Optional, List
from typing import TypeVar, Generic
from markitdown import MarkItDown
import asyncio
import nest_asyncio 
# Add this line to allow nested event loops
nest_asyncio.apply()
import copy
from agent_tools import *
from agent_utils import *

from loguru import logger

config = Config()

# Log to a file with custom timestamp format
logger.add("logs/chain_of_thougth_agent_system.log", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")
model = GeminiModel(config.FLASH2_MODEL)

import logfire

def scrubbing_callback(m: logfire.ScrubMatch):
    if (
        m.path == ('message', 'prompt')
        and m.pattern_match.group(0) == 'Auth'
    ):
        return m.value

    if (
        m.path == ('attributes', 'prompt')
        and m.pattern_match.group(0) == 'Auth'
    ):
        return m.value

    if m.path == ('attributes', 'agent', 'model', 'auth'):
        return m.value

logfire.configure(scrubbing=logfire.ScrubbingOptions(callback=scrubbing_callback))

logfire.instrument_httpx()

/home/dplaia/Projekte/deepresearchagent/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/home/dplaia/Projekte/deepresearchagent/.venv/lib/python3.13/site-packages/pydantic/fields.py:1042: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'env'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warn(
/home/dplaia/Projekte/deepresearchagent/.venv/lib/python3.13/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated

Logfire project URL: ]8;id=219060;https://logfire.pydantic.dev/dplaia/deepresearchagent\https://logfire.pydantic.dev/dplaia/deepresearchagent]8;;\


### Read documents

In [2]:
documents = {}
folder_name = 'input_files/'

# Create directory if it doesn't exist
if not exists(folder_name):
    makedirs(folder_name, exist_ok=True)

main_input_file = join(folder_name, "Vorhabenbeschreibung_NeuroTrust.txt")

if not exists(main_input_file):
    print("file does not exists")
    # Process each file in the input directory
    for filename in listdir(folder_name):
        filepath = join(folder_name, filename)
        
        if not os.path.isfile(filepath):
            continue
            
        try:
            md = MarkItDown()
            result = md.convert(filepath)
            filename = os.path.basename(filepath)
            documents[filename] = result.text_content
        except Exception as e:
            print(f"Error processing {filepath}: {str(e)}")
            continue

    doc = ""
    for filename in documents:
        print(f"Filename: {filename}")
        doc = documents[filename]
        count = word_count(doc)
        print(f"Number of Words in the document: {count}")

        break

    with open(main_input_file, "w", encoding="utf-8") as f:
        f.write(doc)

else:
    with open(main_input_file, "r", encoding="utf-8") as f:
        doc = f.read()

In [3]:
query = f"""Let's assume you have access to the Google Search API and the Google Scholar API. 

Based on the folling document, which search queries would you do, to improve the quality of the document?

# Document: 

{doc}"""

response_thinking = gemini_flash2_thinking_call(query)
console_print(response_thinking.final_answer)

In [ ]:
class ListResponse(BaseModel):
    main_list: list[str] = Field(description="The extracted list.")

#system_prompt="Return a list of things based on user query."

baseAgent = BaseAgent[ListResponse](ListResponse)
result = await baseAgent.run("Give me a list with very small countries of the world.")
print(result.data)

In [ ]:
system_prompt = "Your goal is to extract search queries (e.g., google search, google scholar, etc.) that are mention in a text."
user_input = f"Please extract all search queries from this text: {response_thinking.final_answer}"

baseAgent = BasicAgent[AgentResponse](AgentResponse, system_prompt, config)
result = await baseAgent.run(user_input)

print(result.data)

In [ ]:
# Initialize the chat handler with your credentials and desired model.
chat = ChatHandler(api_key=config.GEMINI_API_KEY, model=config.FLASH2_MODEL_THINKING)
    
response = chat.send_question("What is 2*2?")
print(response)

In [ ]:

class AgentResponse(BaseModel):
    google_search_queries: list[str] = Field(description="The extracted google search queries.")
    google_scholar_queries: list[str] = Field(description="The extracted google scholar queries.")

system_prompt = "Your goal is to extract search queries (e.g., google search, google scholar, etc.) that are mention in a text."
user_input = f"Please extract all search queries from this text: {response_thinking.final_answer}"

agent = Agent(
    model,
    result_type=AgentResponse,
    system_prompt=system_prompt)

result = await agent.run(user_input)
data = result.data

14:38:34.692 agent run prompt=[Scrubbed due to 'auth']
14:38:34.698   preparing model and tools run_step=1
14:38:34.703   model request
14:38:34.751     POST /v1beta/models/gemini-2.0-flash-exp:generateContent


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"


14:38:38.004   handle model response


In [21]:
data.google_scholar_queries

['"neuromorphic computing" "CMOS" "spiking neural networks" review',
 '"spiking neuron" "CMOS" "analog" circuit design',
 '"memristor synapse" "CMOS" implementation',
 '"spike-timing-dependent plasticity" "STDP" "CMOS" implementation',
 '"neuromorphic computing" "energy efficiency" benchmarks comparison',
 '"neuromorphic PUF" "spiking neuron PUF" security analysis',
 '"PUF" "CMOS" "analog variation" entropy',
 '"breakdown structure" "CMOS" "bistable" non-volatile memory',
 '"weightless neural network" "WNN" "PUF" security enhancement',
 '"neuromorphic engineering" "challenges" "future directions"',
 '"VDE SPEC 90033" "neuromorphic computing" "layer model" analysis']

### Setup Agent that reads the document

In [29]:
class DocumentStore:
    """Thread-safe FIFO string storage with non-blocking retrieval
    
    Usage:
        store = DocumentStore()
        store.put('data')
        item = store.get()  # returns None if empty
    """
    def __init__(self):
        self._items = deque()
        self._lock = threading.Lock()

    def put(self, item: str):
        """Add string to storage"""
        with self._lock:
            self._items.append(item)

    def get(self) -> str | None:
        """Retrieve and remove oldest string, returns None if empty"""
        with self._lock:
            return self._items.popleft() if self._items else None

    def clear(self):
        self._items.clear()

    @property
    def count(self) -> int:
        return len(self._items)
        
    def __copy__(self):
        """Create a shallow copy of the DocumentStore instance."""
        new_store = DocumentStore()
        with self._lock:
            new_store._items = deque(self._items)
        return new_store

store = DocumentStore()

In [30]:
system_prompt = read_system_prompt('chain_of_agents')

class AgentResponse(BaseModel):
    main_findings: str = Field(..., description="Markdown summary with inline references (e.g., [Source 1]).")
    agent_instruction: str = Field(..., description="Specific, actionable steps for the next agent (e.g., 'Analyze X in Document 5').")
    links: list[str] = Field(..., description="High-priority links for further research.")
    search_queries_used: list[str] = Field(..., description="Queries executed in this step to avoid repetition.")
    processed_docs: list[str] = Field(..., description="IDs of documents analyzed (e.g., doc_3, doc_7).")
    confidence_score: float = Field(1.0, description="0-1 score indicating confidence in findings (1=high).")
    suggestions_for_improvements: str = Field("", description="Feedback on system/prompt issues.")

agent = Agent(
    model,
    deps_type=store,
    result_type=AgentResponse,
    system_prompt=system_prompt)

# @agent.system_prompt
# def add_document_info() -> str:  
#     return f'Number of stored documents {store.count}.'

# Create a RateLimiter instance allowing 5 requests per 10 seconds
#rate_limiter = RateLimiter(rpm=10, window=60.0)

async_call_limiter = AsyncFunctionCallLimiter(num=1)  # Restrict the number of function calls

@agent.tool_plain
@async_call_limiter
async def google_search(search_query: str, 
                        time_span: Optional[TimeSpan] = None, 
                        web_domain: Optional[str] = None) -> Optional[dict] | str:
    """
    Perform a Google search using the Serper API.
    
    Args:
        search_query (str): The search query string.
        time_span (Optional[TimeSpan], optional): The time span. Defaults to None.
            - Allowed:
                - "qdr:h" (for hour)
                - "qdr:d" (for day)
                - "qdr:w" (for week)
                - "qdr:m" (for month)
                - "qdr:y" (for year)
        web_domain (Optional[str], optional): Search inside a web domain (e.g., web_domain="brainchip.com" -> searches only pages with this domain)
    Returns:
        Optional[dict]: The search results.
    """
    print(f"google_search with arguments = ({search_query}, {time_span}, {web_domain})")
    response = await google_general_search_async(search_query, time_span, web_domain)

    return response

@agent.tool_plain
@async_call_limiter
async def scholar_search(search_query: str, num_pages: int = 1) -> dict  | str:
    """Google scholar search using an API.

        Args:
            search_query (str): The search query string.
            num_pages (int): The amount of page results that should be returned (more pages=more results).
        Returns:
            dict: The search results.
        
    """
    print(f"scholar_search with arguments = ({search_query}, {num_pages})")
    response = await google_scholar_search_async(search_query, num_pages)
    return response


In [ ]:
user_input = f"""
The following document (in German) is a preview of a research proposal that we want to submit. 

Please help us to improve the content of the document in the following ways:
- Do research about the topics mentioned in the document.
- Output things that might be relavant and valueble to the report.
- Include information that might add value to the report.
- Analyse new released papers and see if certain things might have changed.

# Research proposal document:

{doc}
"""

pickle_file = "chain_of_agent_output.pkl"
os.system(f'rm {pickle_file}')

result = await agent.run(user_input, deps=store)
data = result.data
print(f"Number of Links: {len(data.links)}")
with open(pickle_file, "wb") as f:
    pickle.dump(result, f)

#### Load pickle file (instead of running model)

In [15]:
if exists(pickle_file):
    with open(pickle_file, "rb") as f:
        result = pickle.load(f)

In [52]:
console_print(result.data.main_findings)

Based on the conducted search, here are some key points to consider for improving the research proposal:           

 • Focus and Clarity: A research proposal should focus on a few key research questions to support the argument that
   the research needs to be conducted. Minor issues should not dominate the narrative                              
   [https://guides.library.charlotte.edu/c.php?g=1218501&p=8912907].                                               
 • Persuasiveness: The proposal should persuade a committee of scholars that the project has merit                 
   [https://www.ssrc.org/publications/the-art-of-writing-proposals/].                                              
 • Adherence to Guidelines: Ensure the proposal follows all instructions and guidelines, including proper document 
   formatting [https://library.tiffin.edu/writingaresearchproposal/commonproposalwritingmistakes].                 
 • Target Audience: Tailor the proposal to the reviewers. Use clear language to highlight the significance and     
   innovation of the work                                                                                          
   [https://www.ashfoundation.org/news/nine-tips-for-crafting-a-winning-research-grant-proposal/].                 
 • Methodology: Clearly explain how the data will be collected and analyzed                                        
   [https://research.com/research/how-to-write-research-methodology].                                              
 • Convincing the Committee: The proposal should convince the committee that you are ready to start the research   
   project [https://guides.library.unt.edu/Dissertation-thesis-proposal].                                          
 • Articulate Plans: The proposal should clearly articulate the plans for the research and convince others of its  
   value [https://libguides.lb.polyu.edu.hk/researchers_library/research_proposal].                                
 • Well-Organized and Readable: The research strategy should be well-organized, visually appealing, and readable   
   [https://www.niaid.nih.gov/grants-contracts/write-research-plan].                                               
 • Key Elements: Include all key elements involved in the research process                                         
   [http://www.drpaulwong.com/how-to-write-a-research-proposal/].                                                  
 • Literature Review: Provide a literature review and persuasive evidence that there is a need for the research    
   [https://library.sacredheart.edu/c.php?g=29803&p=185956].                                                       
 • Innovation: Highlight the innovative aspects of the research. This could include new perspectives,              
   methodologies, or a novel approach to existing problems                                                         
   [https://www.unh.edu/student-success/sites/default/files/media/2022-07/resources-research-proposals.pdf].       
 • Addressing Limitations: If applicable, address the limitations of previous studies                              
   [https://pmc.ncbi.nlm.nih.gov/articles/PMC5618829/].                                                            
 • Use Key Terms: Be direct and use key terms, avoiding lengthy titles                                             
   [https://www.monash.edu/student-academic-success/excel-at-writing/how-to-write/research-proposal/features-of-a-r
   esearch-proposal].                                                                                              
 • Common Mistakes: Common mistakes include a lack of focus, unclear methodology, and not adhering to guidelines.  
   Ensure your proposal avoids these issues.                                                                       

These points should be used when reviewing the provided German document. Based on these points, please provide     
concrete suggestions on how to improve the proposal.

In [ ]:
temp_folder = "temp/"

if not exists(temp_folder):
    makedirs(temp_folder)

headers = {"User-Agent": "Mozilla/5.0"}

files = {}

for k, link in enumerate(data.links):
    filename = join(temp_folder, f"page_{k}.html")
    try:
        makedirs(os.path.dirname(filename), exist_ok=True)
        response = requests.get(link, headers=headers, timeout=10)
        with open(filename, "w", encoding="utf-8") as f:
            f.write(response.text)

        print(f"Saved: {filename}")
        files[basename(filename)] = link

    except requests.exceptions.Timeout:
        print(f"Timeout occurred for {link}.")
        continue

    except IOError as e:
        print(f"Failed to save {filename}: {e}")
        continue

In [ ]:
md = MarkItDown()
min_word_threshold = 1000
max_word_threshold = 250000

for filename in files:
    link = files[filename]
    filepath = join(temp_folder, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        file_content = f.read()
        count = word_count(file_content)

        if count < max_word_threshold:
            if count < min_word_threshold:
                print(f"Word count: {count} -> Use crawl4ai")
                markdown_output = await crawl4ai_website_async(link)
            else:
                markdown_output = md.convert(filepath).text_content

            store.put(markdown_output)    
        

In [40]:
print(store.count)
store2 = copy.copy(store)

26


In [ ]:
print(store2.count)
console_print(store2.get())